In [8]:
!pip install -q transformers peft bitsandbytes accelerate gradio
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os
from google.colab import drive

search_root = "/content/drive/MyDrive"
teacher_zip = None
student_zip = None

for root, dirs, files in os.walk(search_root):
    if "finetuned_teacher_final.zip" in files:
        teacher_zip = os.path.join(root, "finetuned_teacher_final.zip")
    if "student_model_final.zip" in files:
        student_zip = os.path.join(root, "student_model_final.zip")
    if teacher_zip and student_zip:
        break

if teacher_zip:
    print(f" Found Teacher Zip at: {teacher_zip}")

    extract_path_teacher = "/content/teacher_model"
    print(" Unzipping Teacher Model...")
    with zipfile.ZipFile(teacher_zip, 'r') as z:
        z.extractall(extract_path_teacher)
else:
    print(" Error: Could not find 'finetuned_teacher_final.zip' anywhere in your Drive.")

if student_zip:
    print(f" Found Student Zip at: {student_zip}")

    extract_path_student = "/content/student_model"
    print(" Unzipping Student Model...")
    with zipfile.ZipFile(student_zip, 'r') as z:
        z.extractall(extract_path_student)
else:
    print("Warning: Could not find 'student_model_final.zip'. Skipping student unzip.")

print("Done!")

 Found Teacher Zip at: /content/drive/MyDrive/Kaggle_Project/finetuned_teacher_final.zip
⏳ Unzipping Teacher Model...
 Found Student Zip at: /content/drive/MyDrive/Kaggle_Project/student_model_final.zip
 Unzipping Student Model...
Done!


In [ ]:
from huggingface_hub import login

login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print("1️LOADING TEACHER")
base_teacher = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    quantization_config=bnb_config,
    device_map="cuda:0"
)

teacher_model_path = "/content/teacher_model"
teacher_model = PeftModel.from_pretrained(base_teacher, teacher_model_path)
teacher_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

print("Teacher Loaded")


1️LOADING TEACHER


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Teacher Loaded


In [ ]:
print("LOADING STUDENT")

base_student = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct",
    quantization_config=bnb_config,
    device_map="cuda:0"
)

student_model_path = "/content/student_model"
student_model = PeftModel.from_pretrained(base_student, student_model_path)
student_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

print("Student Loaded ")

LOADING STUDENT


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Student Loaded 


In [ ]:
!pip install -q flask flask-cors pyngrok

In [10]:
import torch
import time
import getpass
import os
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

print("Security Check: Enter your ngrok Authtoken below.")
NGROK_AUTH_TOKEN = getpass.getpass("Paste Token Here: ")

if not NGROK_AUTH_TOKEN:
    raise ValueError(" Error: You must enter a token!")

app = Flask(__name__)
CORS(app)
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok.kill()
public_url = ngrok.connect(5000).public_url
print(f" YOUR API URL IS:  {public_url}")
print("IMPORTANT: In your Stitch/HTML code, your API_URL must be:")
print(f'   const API_URL = "{public_url}/generate";')

@app.route('/', methods=['GET'])
def home():
    return " Server is Running! send POST requests to /generate"
@app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    question = data.get('question', '')
    model_choice = data.get('model', 'teacher')
    print(f"Request: '{question}' | Model: {model_choice}")
    if model_choice.lower() == "teacher":
        model, tokenizer = teacher_model, teacher_tokenizer
    else:
        model, tokenizer = student_model, student_tokenizer
    prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    start_time = time.time()

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=1024, temperature=0.7, do_sample=True)
    end_time = time.time()
    response_text = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    latency = f"{end_time - start_time:.2f}"
    return jsonify({
        "answer": response_text,
        "latency": latency,
        "model_used": model_choice
    })
print(" Server is running. Waiting for requests.")
app.run(port=5000)

Security Check: Enter your ngrok Authtoken below.
Paste Token Here: ··········
 YOUR API URL IS:  https://subcaecal-brian-countably.ngrok-free.dev
IMPORTANT: In your Stitch/HTML code, your API_URL must be:
   const API_URL = "https://subcaecal-brian-countably.ngrok-free.dev/generate";
 Server is running. Waiting for requests.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Jan/2026 07:18:38] "OPTIONS /generate HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Request: 'what is overfitting? explain it clearly' | Model: teacher


INFO:werkzeug:127.0.0.1 - - [26/Jan/2026 07:19:31] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jan/2026 07:21:13] "OPTIONS /generate HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Request: 'what is overfitting? explain it clearly' | Model: student


...

INFO:werkzeug:127.0.0.1 - - [26/Jan/2026 07:26:58] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jan/2026 07:28:46] "OPTIONS /generate HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Request: 'what is overfitting? explain it clearly' | Model: teacher


INFO:werkzeug:127.0.0.1 - - [26/Jan/2026 07:29:47] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jan/2026 07:31:18] "OPTIONS /generate HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Request: 'what is overfitting? explain it clearly' | Model: student


INFO:werkzeug:127.0.0.1 - - [26/Jan/2026 07:32:02] "POST /generate HTTP/1.1" 200 -
